RDD（Resilient Distributed Dataset）叫做弹性分布式数据集，是Spark中最基本的数据抽象，它代表一个不可变、可分区、里面的元素可并行计算的集合。RDD具有数据流模型的特点：自动容错、位置感知性调度和可伸缩性。RDD允许用户在执行多个查询时显式地将工作集缓存在内存中，后续的查询能够重用工作集，这极大地提升了查询速度。

（1）一组分片（Partition），即数据集的基本组成单位。对于RDD来说，每个分片都会被一个计算任务处理，并决定并行计算的粒度。用户可以在创建RDD时指定RDD的分片个数，如果没有指定，那么就会采用默认值。默认值就是程序所分配到的CPU Core的数目。

（2）一个计算每个分区的函数。Spark中RDD的计算是以分片为单位的，每个RDD都会实现compute函数以达到这个目的。compute函数会对迭代器进行复合，不需要保存每次计算的结果。

（3）RDD之间的依赖关系。RDD的每次转换都会生成一个新的RDD，所以RDD之间就会形成类似于流水线一样的前后依赖关系。在部分分区数据丢失时，Spark可以通过这个依赖关系重新计算丢失的分区数据，而不是对RDD的所有分区进行重新计算。

（4）一个Partitioner，即RDD的分片函数。当前Spark中实现了两种类型的分片函数，一个是基于哈希的HashPartitioner，另外一个是基于范围的RangePartitioner。只有对于于key-value的RDD，才会有Partitioner，非key-value的RDD的Parititioner的值是None。Partitioner函数不但决定了RDD本身的分片数量，也决定了parent RDD Shuffle输出时的分片数量。

（5）一个列表，存储存取每个Partition的优先位置（preferred location）。对于一个HDFS文件来说，这个列表保存的就是每个Partition所在的块的位置。按照“移动数据不如移动计算”的理念，Spark在进行任务调度的时候，会尽可能地将计算任务分配到其所要处理数据块的存储位置。

使用手册 
http://spark.apache.org/docs/latest/api/python/pyspark.html


In [1]:
#pyspark.SparkContext()是spark应用的入口，也可以称为驱动
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setAppName("sparkApp1").setMaster("local")
sc = SparkContext.getOrCreate(conf)

In [3]:
#parallelize（c，numSlices=None）分发本地Python集合以形成RDD。如果输入表示性能范围，则建议使用xrange。
#glom（）通过将每个分区内的所有元素合并到一个列表中返回一个RDD。
rdd1 = sc.parallelize([0,2,3,4,6], 5)
rdd2 = sc.parallelize(range(0, 6, 2), 5)
print(rdd1.collect())
print(rdd1.glom().collect())
print(rdd2.collect())
print(rdd2.glom().collect())

[0, 2, 3, 4, 6]
[[0], [2], [3], [4], [6]]
[0, 2, 4]
[[], [0], [], [2], [4]]


In [4]:
#runJob(rdd, partitionFunc, partitions=None, allowLocal=False）
#在指定的分区集合上执行给定的分区，将结果作为元素的数组返回。如果没有指定分区，那么它将在所有分区上运行。
sc.runJob(rdd1, lambda part: [x * x for x in part], [0, 2], True)

[0, 9]

In [6]:
print(sc.startTime)
print(sc.sparkUser())

1528077753028
ffzs


In [ ]:
# rdd.glom()
# glom()定义了将原rdd相同分区的元素放在一个列表中构成新的rdd的转换操作。
# rdd.collect()
# 返回由rdd元素组成的列表
# rdd.collectAsMap()
# 将键值对形式的RDD以字典的形式返回给master 

In [37]:
# cache()
# 将RDD持久化为MEMORY_ONLY

In [34]:
# map(f, preservesPartitioning=False)
# 通过对这个RDD的每个元素应用一个函数来返回一个新的RDD
rdd = sc.parallelize(["b", "a", "c"])
sorted(rdd.map(lambda x:(x, x*2, 1)).collect())

[('a', 'aa', 1), ('b', 'bb', 1), ('c', 'cc', 1)]

In [38]:
#flatMap(f, preservesPartitioning=False)
#首先将一个函数应用到这个RDD的所有元素上，然后将结果全部展开，返回一个新的RDD
rdd = sc.parallelize([2, 3, 4])
print(sorted(rdd.flatMap(lambda x: range(1, x)).collect()))
print(sorted(rdd.map(lambda x: [(x, x), (x, x)]).collect()))
print(sorted(rdd.flatMap(lambda x: [(x, x), (x, x)]).collect()))


[1, 1, 1, 2, 2, 3]
[[(2, 2), (2, 2)], [(3, 3), (3, 3)], [(4, 4), (4, 4)]]
[(2, 2), (2, 2), (3, 3), (3, 3), (4, 4), (4, 4)]


In [39]:
# mapValues(f)
#通过map函数对RDD中的每个key传递value，而不改变键;同时保留了原始的RDD分区。
x = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
def f(x): return len(x)
x.mapValues(f).collect()

[('a', 3), ('b', 1)]

In [52]:
#flatMapValues(f)
#通过flatMap函数传递键值对RDD中的每个值，而不改变键;这也保留了原始的RDD分区。
x = sc.parallelize([("a", ["x", "y", "z"]), ("b", ["p", "r"])])
def f(x): return x
print(x.flatMapValues(f).collect())
print(x.mapValues(f).collect())

[('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]
[('a', ['x', 'y', 'z']), ('b', ['p', 'r'])]


In [53]:
# mapPartitions(f, preservesPartitioning=False)
# 与map不同，map是对每一个元素用函数作用；而mapPartitions是对每一个分区用一个函数去作用，每一个分区的元素先构成一个迭代器iterator，iterator是一个像列表，但里面的元素又保持分布式特点的一类对象;输入的参数就是这个iterator，然后对iterator进行运算，iterator支持的函数不是太多，sum,count等一些spark定义的基本函数应该都是支持的。但如果要进行更为复杂的一些个性化函数运算，可以就用不了。实践中发生可以通过[x for i in iterator]的方式，将iterator转换为列表，然后就可以进行各种操作。但是这样在分区内部或分组内部就失去了分布式运算的特点。
# yield是生成的意思，但是在python中则是作为生成器理解，生成器的用处主要可以迭代，这样简化了很多运算模型。
rdd = sc.parallelize([1, 2, 3, 4], 2)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f).collect()

[3, 7]

In [55]:
# mapPartitionsWithIndex(f, preservesPartitioning=False)
# 通过在这个RDD的每个分区上应用一个函数来返回一个新的RDD，同时跟踪原始分区的索引。为对索引进行操作提供可能
rdd = sc.parallelize([1, 2, 3, 4], 4)
def f(splitIndex, iterator): yield splitIndex
rdd.mapPartitionsWithIndex(f).sum()

6

In [57]:
# partitionBy(numPartitions, partitionFunc=<function portable_hash>)
# 返回使用指定的分区器分区的RDD的副本
# set().intersection 取交集
pairs = sc.parallelize([1, 2, 3, 4, 2, 4, 1]).map(lambda x: (x, x))
sets = pairs.partitionBy(2).glom().collect()
len(set(sets[0]).intersection(set(sets[1])))

0

In [63]:
# coalesce(numPartitions, shuffle=False)
# 返回一个新的RDD，将RDD重新分区,减少分区不适用shuffle ，正加分区数的话要shuffle为true 同repartition
print(sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect())
print(sc.parallelize([1, 2, 3, 4, 5], 3).coalesce(5,True).glom().collect())

[[1], [2, 3], [4, 5]]
[[], [1], [4, 5], [2, 3], []]


In [64]:
# repartition(numPartitions)
# 重新分区，默认shuffle 减少分区用coalesce
rdd = sc.parallelize([1,2,3,4,5,6,7], 4)
print(len(rdd.repartition(2).glom().collect()))
print(len(rdd.repartition(10).glom().collect()))

2
10


In [65]:
# zip(other)
# 一个RDD作为key，另一个让RDD作为value
x = sc.parallelize(range(0,5))
y = sc.parallelize(range(1000, 1005))
x.zip(y).collect()

[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]

In [68]:
# rdd.zipWithIndex()
# RDD为key 排序位置索引作为value
sc.parallelize(["a", "b", "c", "d"], 2).zipWithIndex().collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3)]

In [73]:
# zipWithUniqueId(）
# 根据分区k 按公式k，n+k，2*n+k产生value，RDD为key
rdd = sc.parallelize(["a", "b", "c", "d", "e"], 3)
print(rdd.collect())
print(rdd.glom().collect())
print(rdd.zipWithUniqueId().collect())

['a', 'b', 'c', 'd', 'e']
[['a'], ['b', 'c'], ['d', 'e']]
[('a', 0), ('b', 1), ('c', 4), ('d', 2), ('e', 5)]


In [4]:
# rdd.keyBy()
# RDD通过函数创建元组
x = sc.parallelize(range(0,3)).keyBy(lambda x: x*x)
y = sc.parallelize(zip(range(0,5), range(0,5)))
print(x.collect())
print(y.collect())
[(x, list(map(list, y))) for x, y in sorted(x.cogroup(y).collect())]

[(0, 0), (1, 1), (4, 2)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]


[(0, [[0], [0]]),
 (1, [[1], [1]]),
 (2, [[], [2]]),
 (3, [[], [3]]),
 (4, [[2], [4]])]

In [6]:
# foreach(f)
# 是一个公式作用于rdd所有元素，生成非rdd
def fun(x): 
    print(x)
sc.parallelize([1, 2, 3, 4, 5]).foreach(fun)

In [7]:
# foreachPartition(f)
# 使一个函数作用于RDD上每一个分区
def fun(iterator):
    for x in iterator:
        print(x)
sc.parallelize([1, 2, 3, 4, 5]).foreachPartition(fun)

In [8]:
inputData=sc.parallelize([1,2,3])
def f(x):#定义一个将内容追加于文件末尾的函数
    with open('./example.txt','a+') as fl:
        print(x,file=fl)

open('./example.txt','w').close()#操作之前先关闭之前可能存在的对该文件的写操作
y=inputData.foreach(f)
print(y)
#结果为：None,因为函数f没有返回值
#查看写文件的结果
with open('./example.txt') as fl:
    print(fl.read())

None
1
2
3



'\n1\n2\n3\n'

In [9]:
# groupBy(f, numPartitions=None, partitionFunc=<function portable_hash>）
# 根据函数符合条件与否进行分组返回分组项目的RDD
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x: x % 2).collect()
sorted([(x, sorted(y)) for (x, y) in result])

[(0, [2, 8]), (1, [1, 1, 3, 5])]

In [11]:
#groupByKey(numPartitions=None, partitionFunc=<function portable_hash>)
#原rdd为键值对，groupByKey()则将原rdd的元素相同键的值编进一个sequence
#如果您正在进行分组以执行每个密钥的聚合（例如总计或平均值），则使用reduceByKey或aggregateByKey将提供更好的性能。
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
print(sorted(rdd.groupByKey().mapValues(len).collect()))
print(sorted(rdd.groupByKey().mapValues(list).collect()))

[('a', 2), ('b', 1)]
[('a', [1, 1]), ('b', [1])]


In [20]:
# cogroup(other, numPartitions=None)
# 对于self或other中的每个关键字k，返回一个包含一个元组的结果RDD，以及该关键字在自身和其他关键字中的值列表。
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
print([(x, tuple(map(list, y))) for x, y in sorted(list(x.cogroup(y).collect()))])
print(tuple(map(list,list(x.cogroup(y).collect()[0][1]))))


[('a', ([1], [2])), ('b', ([4], []))]
([4], [])


In [21]:
# groupWith(other, *others)
# cogroup的别名，但支持多个RDD
w = sc.parallelize([("a", 5), ("b", 6)])
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
z = sc.parallelize([("b", 42)])
print([(x, tuple(map(list, y))) for x, y in sorted(list(w.groupWith(x, y, z).collect()))])

[('a', ([5], [1], [2], [])), ('b', ([6], [4], [], [42]))]


In [24]:
# reduce(f)
# reduce函数是将rdd中的每个元素两两之间按函数f进行操作，然后再结果再两两之间按f进行操作，一直进行下去，
# 即所谓的shuffle过程。reduce得到的结果是普通的python对象，而不是rdd.
# operator 操作函数 https://docs.python.org/3/library/operator.html
from operator import *
print(sc.parallelize([1, 2, 3, 4, 5]).reduce(add))
print(sc.parallelize(["a", "b", "c", "d", "e"]).reduce(concat))

15
abcde


In [29]:
# reduceByKey(func, numPartitions=None, partitionFunc=<function portable_hash>)
# 按key分组 组内进行reduce处理
from operator import *
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())

[('a', 2), ('b', 1)]

In [36]:
# reduceByKeyLocally(func)
# 其他与reduceByKey一样，只不过聚合后立即将键，值对以字典的形式传给到集群master，即输出为字典
# 这还将在将结果发送到reducer之前在每个映射器上进行本地合并，类似于“合并器”中的MapReduce的
from operator import *
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
print(rdd.reduceByKeyLocally(add))
print(sorted(rdd.reduceByKeyLocally(add).items()))

{'a': 2, 'b': 1}
[('a', 2), ('b', 1)]


In [38]:
# treeReduce(f, depth=2)
# 分区间多次进行reduce
# depth 树的深度（执行次数？）
add = lambda x, y: x + y
rdd = sc.parallelize([-5, -4, -3, -2, -1, 1, 2, 3, 4], 10)
rdd.treeReduce(add, 2)

-5

In [28]:
# rdd.keys()
# 原rdd的元素为键值对，返回原rdd元素的键为元素的rdd
# rdd.values()
# 原rdd的元素为键值对，返回原rdd元素的值为元素的rdd
w = sc.parallelize([("a", 5), ("b", 6)])
print(w.keys().collect())
print(w.values().collect())

[5, 6]
['a', 'b']


`aggregate函数`

将每个分区里面的元素进行聚合，然后用combine函数将每个分区的结果和初始值(zeroValue)进行combine操作。这个函数最终返回的类型不需要和RDD中元素类型一致。

seqOp操作会聚合各分区中的元素，然后combOp操作把所有分区的聚合结果再次聚合，两个操作的初始值都是zeroValue.  seqOp的操作是遍历分区中的所有元素(T)，第一个T跟zeroValue做操作，结果再作为与第二个T做操作的zeroValue，直到遍历完整个分区。combOp操作是把各分区聚合的结果，再聚合。aggregate函数返回一个跟RDD不同类型的值。因此，需要一个操作seqOp来把分区中的元素T合并成一个U，另外一个操作combOp把所有U聚合。


In [38]:
#aggregate(zeroValue, seqOp, combOp)
seqOp = (lambda x, y : (x[0] + y, x[1] + 1))
combOp = (lambda x, y : (x[0] + y[0], x[1] + y[1]))
print(sc.parallelize([1, 2, 3, 4]).aggregate((0, 0), seqOp, combOp))
print(sc.parallelize([1, 2, 3, 4],3).aggregate((0, 0), seqOp, combOp))
# 三个分区多加了4个6 ?
print(sc.parallelize([1, 2, 3, 4],3).aggregate((0, 6), seqOp, combOp))

(10, 4)
(10, 4)
(10, 28)


In [ ]:
# aggregateByKey(zeroValue, seqFunc, combFunc, numPartitions=None, partitionFunc=<function portable_hash>)
# 跟aggregate逻辑相同，bykey顾名思义 按照key分区 ，而aggregate按区分配；
# 但是zeroValue与aggregate中的用法很不一样，这里的zeroValue是一个值，它即可以跟这样键聚合，也可以跟那个键聚合，而且zeroValue必须与键内聚合时定义的形式一致。

In [37]:
# treeAggregate(zeroValue, seqOp, combOp, depth=2)
# 与aggregate不同的地方是:在每个分区,会做两次或者多次combOp,避免将所有局部的值传给driver端.另外,经过测验初始值zeroValue不会参与combOp.
# depth：树的深度
add = lambda x, y: x + y
rdd = sc.parallelize([-5, -4, -3, -2, -1, 1, 2, 3, 4], 10)
rdd.treeAggregate(0, add, add, 2)

-5

In [39]:
# fold(zeroValue, op)
# partitionBy的简易版，初始一个值，分区内部执行函数和汇总函数为同一个函数
from operator import add
sc.parallelize([1, 2, 3, 4, 5]).fold(0, add)

15

In [40]:
# foldByKey(zeroValue, func, numPartitions=None, partitionFunc=)
# 跟fold逻辑相同，只不过是按照key进行分组
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
from operator import add
sorted(rdd.foldByKey(0, add).collect())

[('a', 2), ('b', 1)]

In [42]:
# combineByKey(createCombiner, mergeValue, mergeCombiners, numPartitions=None, partitionFunc=)
# 将RDD [（K，V）]转换为RDD [（K，C）]类型的结果，通过三个函数进行转换聚合的目的，
# createcombiner函数 rdd值、类型转换
# 根据key对值进行合并
# 将合并列表，将连个c合并成一个
x=sc.parallelize([('a',1),('a',2),('b',1),('b',3),('c',5),('c',6)])
def to_list(a):
    return [a]
def append(a,b):
    a.append(b)
    return a
def extend(a,b):
    a.extend(b)
    return a
print(x.collect())
print(x.combineByKey(to_list,append,extend).collect())

[('a', 1), ('a', 2), ('b', 1), ('b', 3), ('c', 5), ('c', 6)]
[('a', [1, 2]), ('b', [1, 3]), ('c', [5, 6])]


In [43]:
# rdd.sortBy(keyfunc, ascending=True, numPartitions=None)
# 根据key对应的函数进行排序
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
print(sc.parallelize(tmp).sortBy(lambda x: x[0]).collect())
print(sc.parallelize(tmp).sortBy(lambda x: x[1]).collect())

[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
[('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]


In [45]:
# sortByKey(ascending=True, numPartitions=None, keyfunc=<function RDD.<lambda>>)
# 对此RDD进行排序，假定它由（键，值）对组成
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
print(sc.parallelize(tmp).sortByKey().first())
print(sc.parallelize(tmp).sortByKey(True, 1).collect())
print(sc.parallelize(tmp).sortByKey(True, 2).collect())
tmp2 = [('Mary', 1), ('had', 2), ('a', 3), ('little', 4), ('lamb', 5)]
tmp2.extend([('whose', 6), ('fleece', 7), ('was', 8), ('white', 9)])
print(sc.parallelize(tmp2).sortByKey(True, 3, keyfunc=lambda k: k.lower()).collect())

('1', 3)
[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
[('a', 3), ('fleece', 7), ('had', 2), ('lamb', 5), ('little', 4), ('Mary', 1), ('was', 8), ('white', 9), ('whose', 6)]


In [47]:
# stats()
# 计算rdd中全体元素的均值、方差、最大值、最小值和个数的信息
samp=sc.parallelize([1,2,3,4]).stats()
print(samp)

(count: 4, mean: 2.5, stdev: 1.118033988749895, max: 4.0, min: 1.0)


In [48]:
# rdd.count()
# 计算rdd所有元素个数
sc.parallelize([2, 3, 4]).count()

3

In [3]:
# countApprox(timeout, confidence=0.95)
# 在限定时间内做出有可能的结果，即使任务没有完成
rdd = sc.parallelize(range(10000), 10)
rdd.countApprox(1000, 1.0)

10000

In [5]:
# countApproxDistinct(relativeSD=0.05)
# 返回RDD中不同值数的近似值
# relativeSD 相对准确度。较小的值创建需要更多空间的计数器。它必须大于0.000017。
n = sc.parallelize(range(1000)).map(str).countApproxDistinct()
print(n)
n = sc.parallelize([i % 20 for i in range(1000)]).countApproxDistinct()
print(n)

1060
19


In [9]:
# countByKey()
# 计算每个键的元素数量，并将结果作为字典返回给主数据。
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
print(sorted(rdd.countByKey().items()))
print(rdd.countByKey())

[('a', 2), ('b', 1)]
defaultdict(<class 'int'>, {'a': 2, 'b': 1})


In [21]:
# countByValue()
# 将此RDD中每个唯一值的计数返回为（值，计数）对的字典。
print(sc.parallelize([1, 2, 1, 2, 2], 2).glom().collect())
print(sorted(sc.parallelize([1, 2, 1, 2, 2], 2).countByValue().items()))

[[1, 2], [1, 2, 2]]
[(1, 2), (2, 3)]


In [13]:
# first() 返回第一个元素
# max()返回最大值
# take(num) 返回开始num个值
# top(num, key=None) 计算rdd所有元素按降序排列后最顶部的几个元素
# min() rdd中的最小值
# mean() 计算rdd所有元素均值
# variance() 方差
# stdev() 标准差
# sum() 和

In [19]:
# histogram(buckets）
# 对rdd中的元素进行频数统计，统计区间有两种，一种是给出段数，一种是直接给出区间。返回为元组
rdd = sc.parallelize(range(51))
print(rdd.histogram(2))
print(rdd.histogram([0, 5, 25, 50]))
print(rdd.histogram([0, 15, 30, 45, 60]))
rdd = sc.parallelize(["ab", "ac", "b", "bd", "ef"])
print(rdd.histogram(("a", "b", "c")))

([0, 25, 50], [25, 26])
([0, 5, 25, 50], [5, 20, 26])
([0, 15, 30, 45, 60], [15, 15, 15, 6])
(('a', 'b', 'c'), [2, 2])


In [20]:
# pipe(command, env=None, checkCode=False)
# 通过管道向后面环节输出command处理过的结果，具体功能就体现在command，command为linux命令。 
# pipe函数中的'cat'为linux命令，表示打印内容。
sc.parallelize(['1', '2', '', '3']).pipe('cat').collect()

['1', '2', '', '3']

In [21]:
# filter(f)
# 返回满足条件的新RDD
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

In [10]:
# distinct(numPartitions=None)
# 返回一个没有重复元素的新RDD，就是去重处理
sorted(sc.parallelize([1, 1, 2, 3]).distinct().collect())

[1, 2, 3]

In [3]:
# sorted(sc.parallelize([1, 1, 2, 3]).distinct().collect())
# 返回此RDD的采样子集
# withReplacement：是否重复采样
# fraction：样本预期占RDD的大小，每一个元素被取到的概率一样，是一个【0,1】的数
# seed 随机模式的种子
rdd = sc.parallelize(range(100), 4)
print(rdd.sample(False, 0.1, 81).count())
print(rdd.sample(False, 0.2, 81).count())

11
20


In [28]:
# sampleByKey(withReplacement, fractions, seed=None)
# 返回按键取样的RDD的子集（通过分层抽样）。用分数指定的不同键的变量采样率来创建这个RDD的样本，这是抽样速率图的关键。
# 多个key的fractions 以字典方式传递
fractions = {"a": 0.2, "b": 0.1}
rdd = sc.parallelize(fractions.keys()).cartesian(sc.parallelize(range(0, 1000)))
sample = dict(rdd.sampleByKey(False, fractions, 2).groupByKey().collect())
print(rdd.take(10))
print(len(sample['a']), len(sample['b']))
print(sorted(sample['a'])[:10])

[('a', 0), ('b', 0), ('a', 1), ('a', 2), ('b', 1), ('b', 2), ('a', 3), ('a', 4), ('a', 5), ('a', 6)]
209 98


AttributeError: 'ResultIterable' object has no attribute 'takeSample'

In [26]:
# sampleStdev()
# 计算这个RDD元素的样本标准差（通过除以N-1而不是N）来修正估计标准差的偏差。
sc.parallelize([1, 2, 3]).sampleStdev()

1.0

In [27]:
# sampleVariance()
# 计算这个RDD元素的样本方差（它纠正了通过除以N-1而不是N来估计方差的偏差）。
sc.parallelize([1, 2, 3]).sampleVariance()

1.0

In [31]:
# takeSample(withReplacement, num, seed=None)
# 返回这个RDD的一个固定大小的采样子集。
# 只有当结果数组被认为是很小的时候，才应该使用这个方法，因为所有的数据都被加载到驱动程序的内存中。
rdd = sc.parallelize(range(0, 10))
print(rdd.takeSample(True, 12, 1))
print(len(rdd.takeSample(False, 5, 2)))

[6, 9, 9, 8, 0, 7, 0, 8, 3, 6, 7, 8]
5


In [32]:
# toLocalIterator()
# 返回包含这个RDD中所有元素的迭代器。迭代器将消耗与此RDD中最大分区相同的内存。
rdd = sc.parallelize(range(10))
[x for x in rdd.toLocalIterator()]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [39]:
# union(other)
# 返回这个RDD和另一个的结合。不去重
rdd = sc.parallelize([1, 1, 2, 3])
rdd.union(rdd).collect()

[1, 1, 2, 3, 1, 1, 2, 3]

In [40]:
# intersection(other)
# 返回这个RDD和另一个的交集。即使输入RDDs完成了，输出也不会包含任何重复的元素。
# 该方法在内部执行洗牌。
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

[2, 1, 3]

In [41]:
# subtract(other, numPartitions=None)
# 返回自己有其他没有的元素的值
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 3)])
y = sc.parallelize([("a", 3), ("c", None)])
sorted(x.subtract(y).collect())

[('a', 1), ('b', 4), ('b', 5)]

In [42]:
# subtractByKey(other, numPartitions=None)
# 返回每一个（键，值）对，在另一个没有成对的匹配键。
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 2)])
y = sc.parallelize([("a", 3), ("c", None)])
sorted(x.subtractByKey(y).collect())

[('b', 4), ('b', 5)]

In [43]:
# cartesian(other)
# 返回这个RDD和另一个RDD的笛卡尔积，也就是所有成对的元素（a，b）的RDD，a为本身RDD，b为其他RDD
rdd = sc.parallelize([1, 2])
sorted(rdd.cartesian(rdd).collect())

[(1, 1), (1, 2), (2, 1), (2, 2)]

In [44]:
# join(other, numPartitions=None)
# 返回一个包含所有成对元素的RDD，其中包含在self和other中匹配的键。每一对元素都将作为一个（k，（v1，v2））返回，其中（k，v1）为self（k，v2）为other。
# 在集群中执行散列连接
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("a", 3)])
sorted(x.join(y).collect())

[('a', (1, 2)), ('a', (1, 3))]

In [45]:
# rightOuterJoin(other, numPartitions=None)
# 对于在otherRDD中的每一个(k, w)元素，生成的RDD中有ｋ键的生成(k, (v, w)),　如果没有ｋ键的话也要生成none补位(k,(None, w))
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
sorted(y.rightOuterJoin(x).collect())

[('a', (2, 1)), ('b', (None, 4))]

In [46]:
# leftOuterJoin(other, numPartitions=None)
# 就是用第二个rdd的key去第一个rdd中寻找，在value组合的时候还是第一个rdd的值在前，第二个rdd的值在后。其他与leftOuterJoin完全一样。
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
sorted(x.leftOuterJoin(y).collect())

[('a', (1, 2)), ('b', (4, None))]

In [48]:
# randomSplit(weights, seed=None)
# 将RDD按照一定的比例随机分开
rdd = sc.parallelize(range(500), 1)
rdd1, rdd2 = rdd.randomSplit([2, 3], 17)
print(len(rdd1.collect() + rdd2.collect()))
print(rdd1.count(), rdd2.count())

500
192 308
